In [7]:
# import library
import pandas
import numpy as np
import math
import matplotlib.pyplot as plt

import pymoo
from pymoo.model.problem import Problem


import joblib

In [5]:
# import ML model

ML_model_Llt = joblib.load('leakage_inductance.pkl')
ML_model_Lmt = joblib.load('magnetizing_inductance.pkl')
ML_model_Rt = joblib.load('Rt.pkl')
ML_model_Rr = joblib.load('Rr.pkl')

In [20]:

# core loss parameter
core_temp = 70

core_a = 0.6942
core_x = 1.4472
core_y = 2.4769
core_b = 4.7948
core_c = 0.0684
core_d = 4e-4
core_LT = core_b - core_c*core_temp + core_d*core_temp**2



# NSAG-II main

popsize = int(100/5)
offsprings = int(3000/5)

N11 = np.concatenate((8*np.ones(offsprings),9*np.ones(offsprings),10*np.ones(offsprings),11*np.ones(offsprings),12*np.ones(offsprings)), axis=None)
N12 = np.concatenate((8*np.ones(popsize),9*np.ones(popsize),10*np.ones(popsize),11*np.ones(popsize),12*np.ones(popsize)), axis=None)

freq = 30e+3

permeability = 3500

b_N1 = [2,15]
b_w1 = [30,200]
b_l1 = [10,50]
b_l2 = [50,110]
b_h1 = [50,200]
b_space1 = [4,50]
b_space2 = [4,50]
b_space3 = [4,50]
b_space4 = [4,50]
b_coil_width1 = [2,10]
b_coil_width2 = [4,50]
b_move_z1 = [0.5,5]
b_move_z2 = [0.5,5]
b_offset_z1 = [-20,20]
b_offset_z2 = [-20,20]


class MyProblem(Problem):
    def __init__(self, ML_model_Llt, ML_model_Lmt, ML_model_Rt, ML_model_Rr):
        super().__init__(n_var=15,     #number of inputs
                         n_obj=2,     #number of outputs
                         n_constr=40,  #nubmer of constraints
                         xl=np.array([b_N1[0],b_w1[0],b_l1[0],b_l2[0],b_h1[0],b_space1[0],b_space2[0],b_space3[0],b_space4[0],
                         b_coil_width1[0],b_coil_width2[0],b_move_z1[0],b_move_z2[0],b_offset_z1[0],b_offset_z2[0]]), #input lower bounds
                         xu=np.array([b_N1[1],b_w1[1],b_l1[1],b_l2[1],b_h1[1],b_space1[1],b_space2[1],b_space3[1],b_space4[1],
                         b_coil_width1[1],b_coil_width2[1],b_move_z1[1],b_move_z2[1],b_offset_z1[1],b_offset_z2[1]])) #input upper bounds
        self.ML_model_Llt = ML_model_Llt
        self.ML_model_Lmt = ML_model_Lmt
        self.ML_model_Rt = ML_model_Rt
        self.ML_model_Rr = ML_model_Rr




# ["N1","w1","l1","l2","h1","space1","space2","space3","space4","coil_width1","coil_width2","move_z1","move_z2","offset_z1","offset_z2","freq","Llt","Llr"]
    def _evaluate(self, X, out, *args, **kwargs):

        
        # metric dimension
        #try :
        #    X[:,0] = N11
        #    #temp = X[:,0]*X[:,1]
        #except :
        #    X[:,0] = N12

        N1 = X[:,0] 

        w1 = X[:,1] * 1e-3
        l1 = X[:,2] * 1e-3
        l2 = X[:,3] * 1e-3
        h1 = X[:,4] * 1e-3

        space1 = X[:,5] * 1e-3
        space2 = X[:,6] * 1e-3
        space3 = X[:,7] * 1e-3
        space4 = X[:,8] * 1e-3

        coil_width1 = X[:,9] * 1e-3
        coil_width2 = X[:,10] * 1e-3

        move_z1 = X[:,11] * 1e-3
        move_z2 = X[:,12] * 1e-3

        offset_z1 = X[:,13] * 1e-3
        offset_z2 = X[:,14] * 1e-3


        V1 = 1036
        I1 = 100
        I2 = 100

        l = 2*l1 + l2 + space1 + space3 + coil_width1/2 + coil_width2/2
        h = 2*l1 + h1
        wp = 1*w1 + 2*space2 + coil_width1
        ws = 1*w1 + 2*space4 + coil_width2

        hp = N1*coil_width1 + (N1-1)*move_z1 + 2*abs(offset_z1)
        hs = N1*coil_width2 + (N1-1)*move_z2 + 2*abs(offset_z2)

        
        # Lmt
        f1 = self.ML_model_Lmt.predict( X ) # [unit : mH]
        Lmt = f1 * 1e-3 # [unit : H]

        # total volume
        f2 = l * h * wp   # Volume [Unit : m^3]
        f3 = l * h * ws   # Volume [Unit : m^3]

        V = f2
        
        V_core = (2*l1+l2)*h*w1 - l2*h*w1 # Core volume [unit : m^3]

        mag_current = V1/(2*3.141592*freq)/Lmt
        Req = N1**2/Lmt
        flux = N1 * mag_current/Req
        B = flux / (2*w1*l1)
        #B = V1 / (2*l1*w1) / (2*3.141592*freq) / N1

        coreloss = core_a * freq**core_x * B**core_y * core_LT * V_core # Core loss [unit : W]
        f4 = coreloss

        f5 = self.ML_model_Rt.predict( X ) * I1**2 # copperloss_tx [unit : W]
        f6 = self.ML_model_Rr.predict( X ) * I2**2 # copperloss_rx [unit : W]

        f7 = f4+f5*1+f6 # total loss [unit : W]

        f8 = self.ML_model_Llt.predict( X ) # [unit : uH]
        Llt = f8 * 1e-6 # [unit : H]
        #print(np.average(Llt[0]*1e+6))



        #gLlt1 = -(Llt * 1.5 - 16.77e-6) * (Llt * 0.5 - 16.77e-6) / 5e-12 * 10 # Llt constraint
        #gLlt2 = -(Llt * 1.05 - 16.77e-6) * (Llt * 0.95 - 16.77e-6) / 5e-12 * 1 # Llt constraint
        gLlt = -(Llt * 1.02 - 32.375e-6 * 1) * (Llt * 0.98 - 32.375e-6 * 1) / 5e-12 * 10 # Llt constraint
        #print(np.average(gLlt))

        gspace1 = - (space1 - coil_width1 - 20e-3) * (space1 - 20e-3) / 10e-6 # 1차측 w방향
        gspace2 = - (space2 - coil_width2 - 5e-3) * (space2 - 20e-3) / 10e-6 # 2차측 w방향
        gspace3 = - (space1 - coil_width1 - 20e-3) * (space3 - 20e-3) / 10e-6 # 1차측 l방향
        gspace4 = - (space2 - coil_width2 - 5e-3) * (space4 - 20e-3) / 10e-6 # 2차측 l방향


        gl = -(l-0e-3)*(l-250e-3)*250e-3*1e-3 / 250e-6

        gh = -(h-0e-3)*(h-160e-3)*100e-3*1e-3 / 160e-6
        ghp = -(h-hp) / 100e-3
        ghs = -(h-hs) / 100e-3


        gwp = -(wp-0e-3)*(wp-250e-3)*250e-3*1e-3 / 250e-6
        gws = -(ws-0e-3)*(ws-250e-3)*250e-3*1e-3 / 250e-6

        gB = -(B-0)*(B-0.3) / 0.3 / 0.3

        gV = -(f2-f3) / 1e-3
        print(np.average(B))

        
        """
        g1 = g_N1
        g2 = gLlt3
        g1 = gspace1
        g2 = gspace2
        g3 = gspace3
        g4 = gspace4
        g5 = gh11
        g6 = gh12
        g7 = gl
        g8 = gh
        g9 = gw
        g10 = gB
        g11= g_ins1
        g12 = g_ins2
        g13 = g_ins3
        g14 = g_ins4
        g15 = g_air1
        g16 = g_air2
        g17 = g_air3
        g18 = g_air4
        """

        #g_space = [g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18]
        #g_space = [g1.T,g2.T,g3.T,g4.T,g5.T,g6.T,g7.T,g8.T,g9.T,g10.T,g11.T,g12.T,g13.T,g14.T,g15.T,g16.T,g17.T,g18.T]
        #g_space = np.swapaxes(g_space,0,1)
        #g_space = np.where(g_space<0 , 0 , g_space)
        #vec = np.vectorize(math.clamp(X,0))

        #print(g_space[0])


        out["F"] = np.column_stack([V, f7]) # "Minimize" values (volume, coreloss)
        out["G"] = np.column_stack([gLlt, gspace1, gspace2, gspace3, gspace4, gl, gh, ghp, ghs, gwp, gws, gB, gV])
        
        out["G"] = - out["G"] # Actually < 0 

problem = MyProblem(ML_model_Llt, ML_model_Lmt, ML_model_Rt, ML_model_Rr)


In [16]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover

# ["N1","w1","l1","l2","h1","space1","space2","space3","space4","coil_width1","coil_width2","move_z1","move_z2","offset_z1","offset_z2","freq","Llt","Llr"]
mask = ["int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int"]

algorithm = NSGA2(
    pop_size=100,
    n_offsprings=3000,

    sampling = MixedVariableSampling(mask, {
        "int": get_sampling("int_random")
    }),

    crossover = MixedVariableCrossover(mask, {
        "int": get_crossover("int_sbx", prob=0.7, eta=10)
    }),

    #mutation=get_mutation("real_pm", eta=40),
    mutation = MixedVariableMutation(mask, {
        "int": get_mutation("int_pm", eta=1)
    }),

    eliminate_duplicates=True
)

In [17]:
from pymoo.factory import get_termination

termination = get_termination("n_gen", 500)

In [21]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=15, #RANDOM SEED
               save_history=False,
               verbose=True)
res

#print("Best solution found: %s" % res.X)
#print("Function value: %s" % res.F)
#print("Constraint violation: %s" % res.CV)

0.2575233363628492
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     100 |  0.080031466 |  9.56736E+02 |       1 |            - |            -
0.1727595684405631


TypeError: loop of ufunc does not support argument 0 of type float which has no callable sqrt method